## 数据预处理 清洗

In [ ]:
import re
import pandas as pd
import numpy as np
import jieba
import nltk
import matplotlib.pyplot as plt

In [18]:
df=pd.read_csv('./data.csv')
df.head()

,标题,地址,工资,发布日期,工作经验,学历,公司标签,公司名字,经营类型,公司纬度,公司经度,公司规模,公司福利
0,大数据开发工程师,上海·浦东新区,2-3万,2023-05-18 10:10:16,5-7年,本科,计算机软件,丰盛信息科技（上海）有限公司,外资（非欧美）,121.556476,31.226450,50-150人,"['5-7年', '本科', 'spark', '大数据', '数据分析', 'python..."
1,医用耗材大数据分析师,武汉·武昌区,6千-1.1万,2023-05-18 16:10:44,1年,本科,计算机服务(系统、数据服务、维修),武汉麦迪维斯技术有限公司,民营,114.335941,30.584907,少于50人,"['1年', '本科']"
2,大数据分析师,北京·海淀区,8千-1.2万,2023-05-18 18:09:19,1年,大专,电子技术/半导体/集成电路,京城宏图（北京）科技有限公司,合资,116.295057,39.966992,NaN,"['1年', '大专', '数据分析', 'SPSS', 'R语言', 'C+', '五险一..."
3,大数据开发工程师,北京·海淀区,8千-1.5万,2023-05-18 09:08:20,1年,大专,计算机软件,点奇互联（北京）科技有限公司,民营,116.306164,40.056660,NaN,"['1年', '大专', 'java', '数据库', 'spring', 'mybatis..."
4,金融大数据分析师,北京,9千-1.5万,2023-05-18 18:08:30,1年,大专,快速消费品(食品、饮料、化妆品),深圳市白牛贸易有限公司,民营,116.329937,39.910646,NaN,"['1年', '大专', '数据分析', 'SPSS', 'C+', 'R语言', '绩效奖..."


In [19]:
df.info() #查看数据集的基本信息

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   标题      10000 non-null  object 
 1   地址      10000 non-null  object 
 2   工资      9964 non-null   object 
 3   发布日期    10000 non-null  object 
 4   工作经验    9993 non-null   object 
 5   学历      9686 non-null   object 
 6   公司标签    9996 non-null   object 
 7   公司名字    10000 non-null  object 
 8   经营类型    9992 non-null   object 
 9   公司纬度    10000 non-null  float64
 10  公司经度    10000 non-null  float64
 11  公司规模    9300 non-null   object 
 12  公司福利    10000 non-null  object 
dtypes: float64(2), object(11)
memory usage: 1015.8+ KB


# 数据的预处理
'''
1.对缺失值进行剔除
2.对无效信息列进行删除
3.对地址列进行分割：省,市,区
5.对公司工资进行统一平均
6.对公司福利进行关键词提取和词频统计
'''

In [20]:
# 处理缺失值
# df['工资'].fillna('无',inplace=True)
# df['工作经验'].fillna('无',inplace=True)
# df['学历'].fillna('无',inplace=True)
# df['公司标签'].fillna('无',inpalce=True)
# df['经营类型'].fillna('无',inplace=True)
# df['公司规模'].fillna('无',inplace=True)
df=df.dropna(axis=0)
df.shape[0]

8976

In [21]:
# 处理无效列
del df['发布日期']
del df['公司纬度']
del df['公司经度']

In [22]:
# 处理地址
df[['城市','城区']]=df['地址'].str.split('·',2,expand=True)
df.head()

C:\Users\niki0\AppData\Local\Temp\ipykernel_4936\749655897.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df[['城市','城区']]=df['地址'].str.split('·',2,expand=True)


,标题,地址,工资,工作经验,学历,公司标签,公司名字,经营类型,公司规模,公司福利,城市,城区
0,大数据开发工程师,上海·浦东新区,2-3万,5-7年,本科,计算机软件,丰盛信息科技（上海）有限公司,外资（非欧美）,50-150人,"['5-7年', '本科', 'spark', '大数据', '数据分析', 'python...",上海,浦东新区
1,医用耗材大数据分析师,武汉·武昌区,6千-1.1万,1年,本科,计算机服务(系统、数据服务、维修),武汉麦迪维斯技术有限公司,民营,少于50人,"['1年', '本科']",武汉,武昌区
5,大数据开发工程师,北京·东城区,35-55万/年,5-7年,本科,计算机软件,中远海运科技股份有限公司,国企,1000-5000人,"['5-7年', '本科', '五险一金', '补充医疗保险', '补充公积金', '交通补...",北京,东城区
9,大数据开发工程师,广州·天河区,8千-1.6万,2年,本科,计算机服务(系统、数据服务、维修),广州云杉信息科技有限公司,合资,少于50人,"['2年', '本科', '大数据', '数据库', '数据分析', '周末双休', '五险...",广州,天河区
13,大数据项目经理,北京·东城区,35-55万/年,5-7年,本科,计算机软件,中远海运科技股份有限公司,国企,1000-5000人,"['5-7年', '本科', '五险一金', '补充医疗保险', '补充公积金', '交通补...",北京,东城区


In [23]:
def map_city_to_province(city):
    province_mapping = {
        '上海': '上海',
        '上饶': '江西',
        '东莞': '广东',
        '中卫': '宁夏',
        '中山': '广东',
        '临汾': '山西',
        '临沂': '山东',
        '丽水': '浙江',
        '义乌': '浙江',
        '乌鲁木齐': '新疆',
        '九江': '江西',
        '仙桃': '湖北',
        '佛山': '广东',
        '保定': '河北',
        '信阳': '河南',
        '克拉玛依': '新疆',
        '六安': '安徽',
        '兰州': '甘肃',
        '内江': '四川',
        '北京': '北京',
        '北海': '广西',
        '十堰': '湖北',
        '南京': '江苏',
        '南宁': '广西',
        '南昌': '江西',
        '南通': '江苏',
        '厦门': '福建',
        '合肥': '安徽',
        '咸阳': '陕西',
        '哈尔滨': '黑龙江',
        '唐山': '河北',
        '嘉兴': '浙江',
        '大连': '辽宁',
        '天津': '天津',
        '太原': '山西',
        '孝感': '湖北',
        '宁德': '福建',
        '宁波': '浙江',
        '安顺': '贵州',
        '宜宾': '四川',
        '宜昌': '湖北',
        '宜春': '江西',
        '宝鸡': '陕西',
        '宿迁': '江苏',
        '巴中': '四川',
        '常州': '江苏',
        '常熟': '江苏',
        '广安': '四川',
        '广州': '广东',
        '廊坊': '河北',
        '开封': '河南',
        '张家港': '江苏',
        '徐州': '江苏',
        '德阳': '四川',
        '惠州': '广东',
        '成都': '四川',
        '扬州': '江苏',
        '拉萨': '西藏',
        '无锡': '江苏',
        '昆山': '江苏',
        '昆明': '云南',
        '昌吉': '新疆',
        '晋中': '山西',
        '来宾': '广西',
        '杭州': '浙江',
        '枣庄': '山东',
        '柳州': '广西',
        '株洲': '湖南',
        '榆林': '陕西',
        '武汉': '湖北',
        '永州': '湖南',
        '汕头': '广东',
        '汕尾': '广东',
        '江门': '广东',
        '池州': '安徽',
        '沈阳': '辽宁',
        '沧州': '河北',
        '河源': '广东',
        '泰州': '江苏',
        '泸州': '四川',
        '洛阳': '河南',
        '济南': '山东',
        '海口': '海南',
        '海宁': '浙江',
        '深圳': '广东',
        '清远': '广东',
        '温州': '浙江',
        '渭南': '陕西',
        '湖州': '浙江',
        '湛江': '广东',
        '滁州': '安徽',
        '潍坊': '山东',
        '烟台': '山东',
        '珠海': '广东',
        '白银': '甘肃',
        '百色': '广西',
        '盐城': '江苏',
        '眉山': '四川',
        '石嘴山': '宁夏',
        '石家庄': '河北',
        '福州': '福建',
        '绍兴': '浙江',
        '绵阳': '四川',
        '肇庆': '广东',
        '自贡': '四川',
        '舟山': '浙江',
        '芜湖': '安徽',
        '苏州': '江苏',
        '荆州': '湖北',
        '荆门': '湖北',
        '菏泽': '山东',
        '衢州': '浙江',
        '襄阳': '湖北',
        '西安': '陕西',
        '贵阳': '贵州',
        '赣州': '江西',
        '运城': '山西',
        '连云港': '江苏',
        '遂宁': '四川',
        '郑州': '河南',
        '鄂尔多斯': '内蒙',
        '鄂州': '湖北',
        '重庆': '重庆',
        '金华': '浙江',
        '银川': '宁夏',
        '镇江': '江苏',
        '长春': '吉林',
        '长沙': '湖南',
        '阜阳': '安徽',
        '随州': '湖北',
        '雅安': '四川',
        '青岛': '山东',
        '鞍山': '辽宁',
        '马鞍山': '安徽',
        '黄石': '湖北',
        '黔东南': '贵州',
        '陕西省':'陕西',
        '浙江省':'浙江',
        '江苏省':'江苏',
        '四川省':'四川',
        '广东省':'广东',
        '湖北省':'湖北'
    }
    if province_mapping.get(city):
         province=province_mapping.get(city)
    else:
        province=city
    return province
df['省份'] = df['城市'].apply(map_city_to_province)

In [24]:
# 处理工资
'''
统一将工资规范为 每月平均工资（万元）即 “千/月”。
'''
# df[df['工资']=='无'] # 查看工资缺失的个数
# df['工资'].unique() # 查看工资的唯一性
import re
def normalize_salary(salary):
    try:
        # 类型1： 18-25万/年
        if '年' in salary:
            # 移除单位字符和特殊字符
            salary = re.sub(r'[万千/年·薪]','',salary)
            if '-' in salary:
                start,end = salary.split('-')
                # 月工资 = 年工资 /12
                start = float(start) /12
                end  =  float(end) /12
                # print(start,end)
                # 平均工资 = 范围工资 /2
                salary = (float(start)+float(end)) / 2
                salary = np.around(float(salary),decimals=2) #保留两位小数
                return salary
        else:
            # 类型2: 7-1.2万  6千-2万 1-3万
            if '万' in salary :
                matches = re.sub(r'·\d+薪', '', salary)
                salary = re.sub(r'[万千/年·薪]','',matches)
                start,end = salary.split('-')
                if float(start) > float(end): # 7-1.2万
                    start = float(start) /10 # 千转换为万
                    end = float(end)
                # 平均工资 = 范围工资 /2
                salary = (float(start)+float(end)) / 2
                salary = np.around(float(salary),decimals=2) #保留两位小数
                return salary
            # 类型3： 1-2千
            if '千' in salary and '万' not in salary:
                salary = re.sub(r'[万千/年·薪]','',salary)
                start,end = salary.split('-')
                start = float(start) /10 # 千转换为万
                end = float(end) /10  # 千转换为万
                salary = (float(start)+float(end)) / 2
                salary = np.around(float(salary),decimals=2) #保留两位小数
                return salary
            # 类型4： 200元/天
            if '天' in salary :
                salary = re.findall(r'\d+', salary)[0]
                salary = float(salary) * 30 / 10000 #日结---> 月结  元/天 ---> 万/月
                salary = np.around(float(salary),decimals=2) #保留两位小数
                return salary
            if '无' in salary:
                return None
            else:
                return None
    except:
        return None
df['平均薪水']=df['工资'].astype(str).apply(normalize_salary)
df['平均薪水']=df['平均薪水'].apply(lambda x:x*10)
df['平均薪水'].isnull().sum() #查看缺失情况
df=df.dropna(subset='平均薪水')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8963 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   标题      8963 non-null   object 
 1   地址      8963 non-null   object 
 2   工资      8963 non-null   object 
 3   工作经验    8963 non-null   object 
 4   学历      8963 non-null   object 
 5   公司标签    8963 non-null   object 
 6   公司名字    8963 non-null   object 
 7   经营类型    8963 non-null   object 
 8   公司规模    8963 non-null   object 
 9   公司福利    8963 non-null   object 
 10  城市      8963 non-null   object 
 11  城区      6391 non-null   object 
 12  省份      8963 non-null   object 
 13  平均薪水    8963 non-null   float64
dtypes: float64(1), object(13)
memory usage: 1.0+ MB


In [25]:
# 给薪水划分等级低、中、高
def salary_classification(salary):
    if salary <= 9.5:
        return "低"
    elif salary <= 13.0:
        return "中"
    else:
        return "高"
df['薪水等级']=df['平均薪水'].apply(salary_classification)

In [26]:
# 处理公司经营类型
df['经营类型'].unique()
'''
可知暂不用做什么处理
'''

'\n可知暂不用做什么处理\n'

In [27]:
# 根据公司标签获取行业领域
def map_company_labels(company_labels):
    label_map = {
        "计算机软件": "IT/互联网",
        "计算机服务(系统、数据服务、维修)": "IT/互联网",
        "新能源": "能源",
        "通信/电信/网络设备": "通信/电信",
        "互联网/电子商务": "IT/互联网",
        "快速消费品(食品、饮料、化妆品)": "消费品",
        "电子技术/半导体/集成电路": "电子/半导体",
        "金融/投资/证券": "金融",
        "通信/电信运营、增值服务": "通信/电信",
        "仪器仪表/工业自动化": "工业",
        "交通/运输/物流": "交通/物流",
        "教育/培训/院校": "教育",
        "电气/电力/水利": "能源",
        "机械/设备/重工": "机械",
        "专业服务(咨询、人力资源、财会)": "专业服务",
        "房地产": "房地产",
        "医疗设备/器械": "医疗",
        "多元化业务集团公司": "其他",
        "服装/纺织/皮革": "服装/纺织",
        "汽车": "汽车",
        "保险": "金融",
        "政府/公共事业": "政府/非盈利",
        "非营利组织": "政府/非盈利",
        "批发/零售": "零售",
        "建筑/建材/工程": "建筑",
        "会计/审计": "会计/审计",
        "法律": "法律",
        "影视/媒体/艺术/文化传播": "媒体/娱乐",
        "检测，认证": "其他",
        "航天/航空": "航空/航天",
        "环保": "环保",
        "石油/化工/矿产/地质": "能源",
        "学术/科研": "科研/院校",
        "制药/生物工程": "医疗",
        "银行": "金融",
        "计算机硬件": "IT/互联网",
        "文字媒体/出版": "媒体/娱乐",
        "酒店/旅游": "酒店/旅游",
        "餐饮业": "餐饮业",
        "贸易/进出口": "贸易",
        "物业管理/商业中心": "房地产",
        "农/林/牧/渔": "农业",
        "医疗/护理/卫生": "医疗",
        "原材料和加工": "工业",
        "外包服务": "专业服务",
        "印刷/包装/造纸": "制造",
        "家具/家电/玩具/礼品": "家居/玩具",
        "办公用品及设备": "办公用品",
        "广告": "广告",
        "奢侈品/收藏品/工艺品/珠宝": "奢侈品",
        "生活服务": "服务业",
        "中介服务": "服务业",
        "公关/市场推广/会展": "广告",
        "租赁服务": "服务业",
        "娱乐/休闲/体育": "媒体/娱乐",
        "汽车零配件": "汽车",
        "家居/室内设计/装潢": "家居/玩具",
        "网络游戏": "游戏",
        "美容/保健": "美容/保健",
        "采掘业/冶炼": "采掘/冶炼"
    }
    mapped_label = label_map.get(company_labels)
    return mapped_label
df['行业领域']=df['公司标签'].apply(lambda x:map_company_labels(x))

In [15]:
df.head()

,标题,地址,工资,工作经验,学历,公司标签,公司名字,经营类型,公司规模,公司福利,城市,城区,省份,平均薪水,薪水等级,行业领域
0,大数据开发工程师,上海·浦东新区,2-3万,5-7年,本科,计算机软件,丰盛信息科技（上海）有限公司,外资（非欧美）,50-150人,"['5-7年', '本科', 'spark', '大数据', '数据分析', 'python...",上海,浦东新区,上海,25.0,高,IT/互联网
1,医用耗材大数据分析师,武汉·武昌区,6千-1.1万,1年,本科,计算机服务(系统、数据服务、维修),武汉麦迪维斯技术有限公司,民营,少于50人,"['1年', '本科']",武汉,武昌区,湖北,8.5,低,IT/互联网
5,大数据开发工程师,北京·东城区,35-55万/年,5-7年,本科,计算机软件,中远海运科技股份有限公司,国企,1000-5000人,"['5-7年', '本科', '五险一金', '补充医疗保险', '补充公积金', '交通补...",北京,东城区,北京,37.5,高,IT/互联网
9,大数据开发工程师,广州·天河区,8千-1.6万,2年,本科,计算机服务(系统、数据服务、维修),广州云杉信息科技有限公司,合资,少于50人,"['2年', '本科', '大数据', '数据库', '数据分析', '周末双休', '五险...",广州,天河区,广东,12.0,中,IT/互联网
13,大数据项目经理,北京·东城区,35-55万/年,5-7年,本科,计算机软件,中远海运科技股份有限公司,国企,1000-5000人,"['5-7年', '本科', '五险一金', '补充医疗保险', '补充公积金', '交通补...",北京,东城区,北京,37.5,高,IT/互联网


In [28]:
df.to_csv('./data/df.csv')

In [29]:
df['省份'].unique()

array(['上海', '湖北', '北京', '广东', '重庆', '云南', '四川', '江苏', '安徽', '江西', '天津',
       '山东', '福建', '陕西', '河北', '浙江', '甘肃', '河南', '辽宁', '湖南', '海南', '贵州',
       '山西', '宁夏', '广西', '黑龙江', '西藏', '新疆', '吉林', '内蒙'], dtype=object)